## Calibração de intrísicos usando chessboard oii

- Utiliza a biblioteca OpenCV para obter a matriz de intrísicos da câmera
- Para obtê-la, fazemos uma comparação com distâncias entre pontos conhecidas no mundo real e as distâncias entre pontos no plano da imagem
- É escolhido um plano de calibração no qual conhecemos as distâncias entre os pontos, como por exemplo entre os cantos dos quadrados de um tabuleiro de xadrez

- Para realizar a calibração, é preciso tirar fotos do plano em diferentes poses
- Algumas boas práticas são:
    - Escolha um ambiente iluminado e, se possível, utilize uma fonte luminosa atrás da câmera
    - Prepare um dataset com várias imagens, assim você pode escolher as melhores para realizar a calibração
    - O plano de calibração precisa cobrir a maior parte das fotos, por isso se atente no tamanho do plano que você está utilizando

### Passo a passo da calibração
- Antes de tirar as fotos você precisa de um plano de calibração 
    - Para criar o seu, use o [código](https://docs.opencv.org/4.x/da/d0d/tutorial_camera_calibration_pattern.html) disponibilizado pelo próprio OpenCV
- As imagens utilizadas na calibração precisam estar em escala de cinza!
    - Antes de analisar as imagens, converta-as para grayscale 

- Para encontrar os cantos dos quadrados do tabuleiro, são usadas duas abordagens:
    - A primeira utiliza um método que determina o canto do quadrado no ponto em que o branco se torna preto na imagem, pela função `findChessboardCorners()`. Os cantos encontrados não são tão exatos, por isso passam por um refinamento pela função `cornerSubPix()`
    - A segunda utiliza uma abordagem diferente, na qual usa a função `findChessboardCornersSB()`

### Importação das bibliotecas e informações sobre o padrão de calibração

A depender do padrão que escolher, você deverá mudar:
- patternSize (colunas X linhas)
- squareSize (tamanho do quadrado em mm)
- imgSize (tamanho da imagem)

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np


patternSize = (10,7) # Count the intern rows and colluns
squareSize = 30 # In millimeters 
imgSize = (2592,1944) # In pixels

# USE THE DIRECTORY WHERE YOUR IMAGES ARE! 
images = glob.glob("*jpg")

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## Definir função para reconstrução dos pontos
- Para encontrar os pontos dos cantos do tabuleiro no mundo real, utiliza-se o tamanho dos quadrados e a quantidade de linhas e colunas do tabuleiro

In [ ]:
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize,squareSize)
worldPoints = []
imagePoints = []
worldPointsSB = []
imagePointsSB = [] 

### Abordagem com `findChessboardCorners()` e `cornerSubPix()`

Caso queira ver os cantos dectados pela função, utilize a função `drawChessboardCorners()`

In [ ]:
counter = 0
for fname in images:
    print("=> Processing image {0}".format(fname))
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, patternSize, None)
    if ret == True:
        print("Corners found!")
        cornersRefined = cv2.cornerSubPix(gray, corners, (7,7), (-1,-1), criteria)
        imagePoints.append(cornersRefined)
        worldPoints.append(boardPoints)
        counter+=1
        cv2.drawChessboardCorners(img, patternSize, cornersRefined, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()


### Calibração da câmera

In [ ]:
flagsCalib = cv2.CALIB_RATIONAL_MODEL 

initialCameraMatrix = np.array([[ 1000.,    0., imgSize[0]/2.],
                                 [    0., 1000., imgSize[1]/2.],
                                 [    0.,    0.,           1.]])

initialDistCoeffs = np.zeros((5,1))


ret, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, initialCameraMatrix, initialDistCoeffs,flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", distCoeffs)

### Erro de reprojeção para cada imagem

In [ ]:
print("RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD\n",perViewError)

### Abordagem com `findChessboardCornersSB()`

Caso queira ver os cantos dectados pela função, utilize a função `drawChessboardCorners()`

In [ ]:
counter = 0
for fname in images:
    print("=> Processing image {0}".format(fname))
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, cornersSB = cv2.findChessboardCornersSB(gray, patternSize, cv2.CALIB_CB_EXHAUSTIVE + cv2.CALIB_CB_ACCURACY + cv2.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        print("Corners found!")
        imagePointsSB.append(cornersSB)
        worldPointsSB.append(boardPoints)
        counter+=1
        #cv2.drawChessboardCorners(img, patternSize, cornersSB, ret)
        #cv2.imshow('img', img)
        #cv2.waitKey(1000)
        #cv2.destroyAllWindows()


### Calibração da câmera

In [ ]:
#adicionar novas flags!!!
flagsCalib = cv2.CALIB_RATIONAL_MODEL

initialCameraMatrix = np.array([[ 1000.,    0., imgSize[0]/2.],
                                 [    0., 1000., imgSize[1]/2.],
                                 [    0.,    0.,           1.]])

initialDistCoeffs = np.zeros((5,1))


ret, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewErrorSB = cv2.calibrateCameraExtended(worldPointsSB, imagePointsSB, imgSize, initialCameraMatrix, initialDistCoeffs,flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", distCoeffs)

### Erro de reprojeção para cada imagem

In [ ]:
print("RE-PROJECTION ERROR OF EACH VIEW - CHESSBOARD\n",perViewErrorSB)